### Here, we add derived trade features to features already obtained from the CSV.

In [1]:
%load_ext autoreload
%autoreload 1
%aimport my_functions

import pandas as pd
import datetime as dt
import numpy as np

from my_functions import *

pd.options.mode.chained_assignment = None

In [2]:
'''
Load insider trade data and historic ticker data.
'''
insiderDat = pd.read_csv('data/insiderDat_JuneToAug2021_clean.csv')
historicDat = load_obj('data/historicDat')

In [3]:
DAYS_TO_LOOK_FORWARD = 90
DAYS_TO_LOOK_BACK = 6  # for computing volume volatility and related insider buys
WINDOW_LEN = 3

In [4]:
insiderDat_complete = createAllFeatures(insiderDat, historicDat, DAYS_TO_LOOK_FORWARD,
                                        WINDOW_LEN, DAYS_TO_LOOK_BACK)

In [5]:
print('Here is an example of a row that contains all trade features:\n')
insiderDat_complete.iloc[-1]

Here is an example of a row that contains all trade features:



FilingDate                              2021-08-31
TradeDate                               2021-08-26
Ticker                                        IVFH
CompanyName           Innovative Food Holdings Inc
InsiderName                              Cohn Hank
Title                                          Dir
TradeType                             P - Purchase
Price                                          0.4
Qty                                      3125000.0
Owned                                    3448315.0
DeltaOwn                                 966.54965
Value                                    1250000.0
NumTrades                                        2
TradeToFileTime                                  5
ValueOwned                               1379326.0
VolumeVolatility                          0.810143
PriceVolatility                           0.108384
%FuturePriceChange                       23.910259
Name: 14025, dtype: object

In [6]:
insiderDat_complete.to_csv('data/training_and_cv_data.csv')